In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
%matplotlib inline

In [2]:
all_master = pd.read_csv('data/Master.csv')
all_salaries = pd.read_csv('data/Salaries.csv')
all_appearances = pd.read_csv('data/Appearances.csv')
all_batting = pd.read_csv('data/Batting.csv')
batting = all_batting[['playerID', 'yearID', 'teamID', 
                       'G', 'H', 'BB', 'HBP', 'AB', 'SF']]
batting = batting[batting['SF'] > 0]
batting = batting[batting['G'] > 20]
batting['OBP'] = ((batting['H'] + batting['BB'] + batting['HBP'])/
                  (batting['AB'] + batting['BB'] + batting['HBP'] + batting['SF']))
master = all_master[['playerID', 'nameFirst', 'nameLast']]
appearances = all_appearances[['playerID', 'yearID', 
                               'G_p', 'G_c', 'G_1b', 
                               'G_2b', 'G_3b', 'G_ss', 
                               'G_lf', 'G_cf', 'G_rf']]
salaries = all_salaries[['playerID', 'yearID', 'salary']]
merged = pd.merge(pd.merge(pd.merge(master, batting), salaries), appearances)
merged['worth'] = merged['OBP'] / merged['salary'] * 1000000


In [3]:
positions = ['G_p', 'G_c', 'G_1b', 'G_2b', 'G_3b', 
             'G_ss', 'G_lf', 'G_cf', 'G_rf']
def build_team(year='2014'):
    trimmed = merged[merged['yearID'] == 2014]
    trimmed = trimmed.sort('worth', ascending=False)
    trimmed.index = trimmed['worth']
    team = {}
    for i in trimmed.index:
        for pos in positions:
            if trimmed.loc[i, pos] > 0 and pos not in team:
                team[pos] = [trimmed.loc[i, ['playerID', 
                                             'nameFirst', 'nameLast']]]
                break
    return team

In [6]:
moneyball_team = build_team(year='2001')
print(moneyball_team)

{'G_2b': [playerID     harrijo05
nameFirst         Josh
nameLast      Harrison
Name: 0.675858340092, dtype: object], 'G_3b': [playerID     donaljo02
nameFirst         Josh
nameLast     Donaldson
Name: 0.684892086331, dtype: object], 'G_c': [playerID     norride01
nameFirst        Derek
nameLast        Norris
Name: 0.713948945915, dtype: object], 'G_ss': [playerID     rutlejo01
nameFirst         Josh
nameLast      Rutledge
Name: 0.645593086823, dtype: object], 'G_1b': [playerID     vanslsc01
nameFirst        Scott
nameLast     Van Slyke
Name: 0.760943570027, dtype: object], 'G_cf': [playerID     yelicch01
nameFirst    Christian
nameLast        Yelich
Name: 0.717332007173, dtype: object], 'G_p': [playerID     alvarhe01
nameFirst    Henderson
nameLast       Alvarez
Name: 0.387114256773, dtype: object], 'G_rf': [playerID     hicksaa01
nameFirst        Aaron
nameLast         Hicks
Name: 0.671541231306, dtype: object], 'G_lf': [playerID     dickeco01
nameFirst        Corey
nameLast     Dicke

OBP = (Hits + Bases on Balls(walks) + Hit By Pitch)/(At Bats + Bases on Balls(walks) + Hit By Pitch + Sacrifice Flies)

In [5]:
merged.loc